# Interpretability With Tensorflow On Azure Machine Learning Service

## Overview of Tutorial
This notebook is Part 4 (Explaining Your Model Using Interpretability) of a four part workshop that demonstrates an end-to-end workflow for using Tensorflow on Azure Machine Learning Service. The different components of the workshop are as follows:

- Part 1: [Preparing Data and Model Training](https://github.com/microsoft/bert-stack-overflow/blob/master/1-Training/AzureServiceClassifier_Training.ipynb)
- Part 2: [Inferencing and Deploying a Model](https://github.com/microsoft/bert-stack-overflow/blob/master/2-Inferencing/AzureServiceClassifier_Inferencing.ipynb)
- Part 3: [Setting Up a Pipeline Using MLOps](https://github.com/microsoft/bert-stack-overflow/tree/master/3-ML-Ops)
- Part 4: [Explaining Your Model Interpretability](https://github.com/microsoft/bert-stack-overflow/blob/master/4-Interpretibility/IBMEmployeeAttritionClassifier_Interpretability.ipynb)

**In this specific tutorial, we will cover the following topics:**

- TODO
- TODO

## What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)


## What Is Machine Learning Interpretability?
Interpretability is the ability to explain why your model made the predictions it did. The Azure Machine Learning service offers various interpretability features to help accomplish this task. These features include:

- Feature importance values for both raw and engineered features.
- Interpretability on real-world datasets at scale, during training and inference.
- Interactive visualizations to aid you in the discovery of patterns in data and explanations at training time.

By accurately interpretabiliting your model, it allows you to:

- Use the insights for debugging your model.
- Validate model behavior matches their objectives.
- Check for for bias in the model.
- Build trust in your customers and stakeholders.

![](./images/interpretability-architecture.png)

## Install Azure Machine Learning Python SDK

If you are running this on a Notebook VM, the Azure Machine Learning Python SDK is installed by default. If you are running this locally, you can follow these [instructions](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py) to install it using pip.

This tutorial series requires version 1.0.69 or higher. We can import the Python SDK to ensure it has been properly installed:

In [1]:
import azureml.core

print("Azure Machine Learning Python SDK version:", azureml.core.VERSION)

Azure Machine Learning Python SDK version: 1.0.69


## Install Tensorflow 1.14

We will be using an older version (1.14) for this particular tutorial in the series as Tensorflow 2.0 is not yet supported for Interpretibility on Azure Machine Learning service. If are currently running Tensorflow 2.0, run the code below to downgrade the version.

In [ ]:
%pip uninstall tensorflow-gpu keras --yes
%pip install tensorflow-gpu==1.14

Let's make sure we have the right verison.

In [2]:
import tensorflow as tf
tf.version.VERSION

'1.14.0'

## Connect To Workspace

Just like in the previous tutorials, we will need to connect to a [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py).

The following code will allow you to create a workspace if you don't already have one created. You must have an Azure subscription to create a workspace:

```python
from azureml.core import Workspace
ws = Workspace.create(name='myworkspace',
                      subscription_id='<azure-subscription-id>',
                      resource_group='myresourcegroup',
                      create_resource_group=True,
                      location='eastus2')
```

**If you are running this on a Notebook VM, you can import the existing workspace.**

In [3]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Workspace name: tf-world
Azure region: eastus
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: tf-world


> **Note:** that the above commands reads a config.json file that exists by default within the Notebook VM. If you are running this locally or want to use a different workspace, you must add a config file to your project directory. The config file should have the following schema:

```
    {
        "subscription_id": "<SUBSCRIPTION-ID>",
        "resource_group": "<RESOURCE-GROUP>",
        "workspace_name": "<WORKSPACE-NAME>"
    }
```

## Train Model
For this tutorial, we will be using the *tf.keras module* to train a basic feed forward neural network on the IBM Employee Attrition Dataset. 

**We will start by writing the training script to train our model**

In [4]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def preprocess_data(data):
    '''
    
    '''
    # Dropping Employee count as all values are 1 and hence attrition is independent of this feature
    data = data.drop(['EmployeeCount'], axis=1)
    
    # Dropping Employee Number since it is merely an identifier
    data = data.drop(['EmployeeNumber'], axis=1)
    data = data.drop(['Over18'], axis=1)

    # Since all values are 80
    data = data.drop(['StandardHours'], axis=1)

    # Converting target variables from string to numerical values
    target_map = {'Yes': 1, 'No': 0}
    data["Attrition_numerical"] = data["Attrition"].apply(lambda x: target_map[x])
    target = data["Attrition_numerical"]

    data.drop(['Attrition_numerical', 'Attrition'], axis=1, inplace=True)
    
    # Creating dummy columns for each categorical feature
    categorical = []
    for col, value in data.iteritems():
        if value.dtype == 'object':
            categorical.append(col)

    # Store the numerical columns in a list numerical
    numerical = data.columns.difference(categorical)   

    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocess = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    
    pipeline = make_pipeline(preprocess)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(data, 
                                                        target, 
                                                        test_size=0.2,
                                                        random_state=0,
                                                        stratify=target)
    
    return x_train, x_test, y_train, y_test, pipeline, preprocess
    
# Load and preprocess data
attrition_data = pd.read_csv('./data/data.csv')
x_train, x_test, y_train, y_test, pipeline, preprocess = preprocess_data(attrition_data)

# Transform data
x_train_t = pipeline.fit_transform(x_train)
x_test_t = pipeline.transform(x_test)

# Create model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=16, activation='relu', input_shape=(x_train_t.shape[1],)))
model.add(tf.keras.layers.Dense(units=16, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

# Fit model
model.fit(x_train_t, y_train, epochs=20, verbose=1, batch_size=128, validation_data=(x_test_t, y_test))

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1176 samples, validate on 294 samples
Epoch 1/20
1176/1176 [==============================] - 0s 175us/sample - loss: 0.5150 - acc: 0.8308 - val_loss: 0.4791 - val_acc: 0.8401
Epoch 2/20
1176/1176 [==============================] - 0s 19us/sample - loss: 0.4637 - acc: 0.8376 - val_loss: 0.4502 - val_acc: 0.8401
Epoch 3/20
1176/1176 [==============================] - 0s 16us/sample - loss: 0.4375 - acc: 0.8376 - val_loss: 0.4307 - val_acc: 0.8401
Epoch 4/20
1176/1176 [==============================] - 0s 17us/sample - loss: 0.4187 - acc: 0.8384 - val_loss: 0.4157 - val_acc: 0.8401
Epoch 5/20
1176/1176 [==============================] - 0s 16us/sample - loss: 0.4052 - acc: 0.8384 - val_loss: 0.4067 - val_acc: 0.8401
Epoch 6/20
1176/1176 [==============================] - 0s 16us/sample - loss: 0.3958 - acc: 0.8384 - val_loss: 0.3992 - val_acc: 0.8401
Epoch 7/20
1176/1176 [==============================] - 0s 16us/sample - loss: 0.3879 - acc: 0.8384 - val_loss: 0.3936 - val_acc: 

## Explain Model Locally

We will start by explaining the trained model locally.

**Instantiate the explainer object using trained model.**

In [5]:
from interpret.ext.greybox import DeepExplainer

explainer = DeepExplainer(model,
                          x_train,
                          features=x_train.columns,
                          classes=["STAYING", "LEAVING"], 
                          transformations = preprocess,
                          model_task="classification",
                          is_classifier=True)

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/deep/deep_tf.py:118: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



**Generate local explanations**

In [9]:
# You can pass a specific data point or a group of data points to the explain_local function
# E.g., Explain the first data point in the test set
instance_num = 1
local_explanation = explainer.explain_local(x_test[:instance_num])

sorted_local_importance_values = local_explanation.get_ranked_local_values()
sorted_local_importance_names = local_explanation.get_ranked_local_names()

print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

local importance values: [[[0.02745935619113111, 0.02388463189410146, 0.020942007144913077, 0.02072276084300257, 0.01591142771356663, 0.01487850179481488, 0.012745772539601552, 0.009650989505462348, 0.00963469324633479, 0.008773050403817054, 0.006977637853967165, 0.005644893143529896, 0.005188455890287827, 0.0037072910350359166, 0.003232530595886378, 0.001697204236677791, 0.0016097691412437006, 0.0014667453432869921, 0.0, 0.0, 0.0, -0.0004357534955488518, -0.0017016542948617964, -0.0024226699168295633, -0.005660104808416575, -0.009158367349154066, -0.01248942909341209, -0.012904132188475203, -0.018412580084532233, -0.018687076805216047]], [[0.018687076805216047, 0.018412580084532233, 0.012904132188475203, 0.01248942909341209, 0.009158367349154066, 0.005660104808416575, 0.0024226699168295633, 0.0017016542948617964, 0.0004357534955488518, 0.0, 0.0, 0.0, -0.0014667453432869921, -0.0016097691412437006, -0.001697204236677791, -0.003232530595886378, -0.0037072910350359166, -0.005188455890287

**Generate global explanations**

In [10]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(x_test)

# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

# Per class feature names
print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))

# Per class feature importance values
print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

global importance rank: {'NumCompaniesWorked': 0.03417471731378875, 'EducationField': 0.03277963455957586, 'OverTime': 0.03186986137032637, 'StockOptionLevel': 0.027491102002814863, 'DistanceFromHome': 0.02726175194254244, 'DailyRate': 0.026943844012963694, 'Department': 0.02673539651506227, 'YearsSinceLastPromotion': 0.0242831720212505, 'EnvironmentSatisfaction': 0.023014777976693532, 'JobRole': 0.021821173867686894, 'TotalWorkingYears': 0.021542080085869175, 'TrainingTimesLastYear': 0.020734608450857426, 'YearsWithCurrManager': 0.020032447284073274, 'RelationshipSatisfaction': 0.018775204327377625, 'YearsInCurrentRole': 0.01746664967347312, 'JobInvolvement': 0.014602441754769383, 'MaritalStatus': 0.014237106674383018, 'WorkLifeBalance': 0.014229805121233768, 'JobSatisfaction': 0.01347278631249912, 'JobLevel': 0.012899359471362863, 'Age': 0.01128978592129426, 'HourlyRate': 0.009253448266305697, 'PercentSalaryHike': 0.00885626675733648, 'BusinessTravel': 0.008690116201352011, 'Gender':

**Visualize our explanations**

In [17]:
from interpret_community.widget import ExplanationDashboard
from interpret_community.common.model_wrapper import wrap_model
from interpret_community.dataset.dataset_wrapper import DatasetWrapper
from sklearn.pipeline import Pipeline

wrapped_model, ml_domain = wrap_model(model, DatasetWrapper(x_test_t), "classification")
wrapped_model.fit = model.fit
dashboard_pipeline = Pipeline(steps=[('preprocess', preprocess), ('network', wrapped_model)])
ExplanationDashboard(global_explanation, dashboard_pipeline, datasetX=x_test)

ExplanationWidget(value={'predictedY': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…

## Explain Model On Azure Machine Learning Service

Now let's train our model on Azure Machine Learning service and explain remotely.

**Instead of running our script in the notebook, let's start by writing the script to a train.py file.**

In [ ]:
%%writefile train.py
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def preprocess_data(data):
    '''
    
    '''
    # Dropping Employee count as all values are 1 and hence attrition is independent of this feature
    data = data.drop(['EmployeeCount'], axis=1)
    
    # Dropping Employee Number since it is merely an identifier
    data = data.drop(['EmployeeNumber'], axis=1)
    data = data.drop(['Over18'], axis=1)

    # Since all values are 80
    data = data.drop(['StandardHours'], axis=1)

    # Converting target variables from string to numerical values
    target_map = {'Yes': 1, 'No': 0}
    data["Attrition_numerical"] = data["Attrition"].apply(lambda x: target_map[x])
    target = data["Attrition_numerical"]

    data.drop(['Attrition_numerical', 'Attrition'], axis=1, inplace=True)
    
    # Creating dummy columns for each categorical feature
    categorical = []
    for col, value in data.iteritems():
        if value.dtype == 'object':
            categorical.append(col)

    # Store the numerical columns in a list numerical
    numerical = data.columns.difference(categorical)   

    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocess = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    
    pipeline = make_pipeline(preprocess)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(data, 
                                                        target, 
                                                        test_size=0.2,
                                                        random_state=0,
                                                        stratify=target)
    
    return x_train, x_test, y_train, y_test, pipeline, preprocess
    
# Load and preprocess data
attrition_data = pd.read_csv('./data/data.csv')
x_train, x_test, y_train, y_test, pipeline, preprocess = preprocess_data(attrition_data)

# Transform data
x_train_t = pipeline.fit_transform(x_train)
x_test_t = pipeline.transform(x_test)

# Create model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=16, activation='relu', input_shape=(x_train_t.shape[1],)))
model.add(tf.keras.layers.Dense(units=16, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

# Fit model
model.fit(x_train_t, y_train, epochs=20, verbose=1, batch_size=128, validation_data=(x_test_t, y_test))

# Save model
model.save('./outputs/model.h5')

**Now submit the script to be run on the cluster that was created in tutorial 1**

In [ ]:
from azureml.train.dnn import TensorFlow

compute_target = workspace.compute_targets['v100cluster']

estimator = TensorFlow(source_directory='.',
                        entry_script='train.py',
                        compute_target=compute_target,
                        framework_version='1.13',
                        use_gpu=True)

run = experiment.submit(estimator)

**Monitor the run as usual**

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run1).show()

**Register the trained model which we will use to explain**

In [ ]:
model = run.register_model(model_name='ibm-attrition-classifier', 
                                model_path='./outputs/model.h5',
                                description='IBM Employee Attrition data classifier')

**TODO: Explain on AML Compute**

## Explain A Deployed Model

Now let's deploy our model and explain during runtime.

**TODO**

## Next Steps
Learn about other use cases of the explain package on a:
       
1. [Training time: regression problem](./explain-regression-local.ipynb)
1. [Training time: binary classification problem](./explain-binary-classification-local.ipynb)
1. [Training time: multiclass classification problem](./explain-multiclass-classification-local.ipynb)
1. [Explain models with advanced feature transformations](./advanced-feature-transformations-explain-local.ipynb)
1. [Save model explanations via Azure Machine Learning Run History](../azure-integration/run-history/save-retrieve-explanations-run-history.ipynb)
1. [Run explainers remotely on Azure Machine Learning Compute (AMLCompute)](../azure-integration/remote-explanation/explain-model-on-amlcompute.ipynb)
1. Inferencing time: deploy a classification model and explainer:
    1. [Deploy a locally-trained model and explainer](../azure-integration/scoring-time/train-explain-model-locally-and-deploy.ipynb)
    1. [Deploy a remotely-trained model and explainer](../azure-integration/scoring-time/train-explain-model-on-amlcompute-and-deploy.ipynb)